In [1]:
# %%
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install scikit-learn
%pip install matplotlib

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
import numpy as np

In [3]:
# Darknet-19 model implementation
class Darknet(nn.Module):
    def __init__(self, num_classes):
        super(Darknet, self).__init__()
        # Define the architecture as per Darknet-19
        self.layers = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Layer 2
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Layer 3
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(128, 64, kernel_size=1, stride=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Layer 4
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(256, 128, kernel_size=1, stride=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Layer 5
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 256, kernel_size=1, stride=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 256, kernel_size=1, stride=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Layer 6
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(1024, 512, kernel_size=1, stride=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(1024, 512, kernel_size=1, stride=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            # Layer 7
            nn.Conv2d(1024, num_classes, kernel_size=1, stride=1),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x.view(x.size(0), -1)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
# Define dataset paths
train_data_path = 'C:\\projects\\BloodNets\\blood_data\\train'
test_data_path = 'C:\\projects\\BloodNets\\blood_data\\test'

# Load datasets
train_data = datasets.ImageFolder(train_data_path, transform=transform)
test_data = datasets.ImageFolder(test_data_path, transform=transform)


In [6]:
# Training function
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)

In [7]:
# Testing function
def test(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    metrics = {
        'loss': total_loss / len(loader.dataset),
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds, average='weighted'),
        'recall': recall_score(all_labels, all_preds, average='weighted'),
        'f1_score': f1_score(all_labels, all_preds, average='weighted')
    }
    return metrics

In [8]:
# Initialize K-Fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)



In [9]:
# Initialize Darknet model
model = Darknet(num_classes=8)
model = model.to(device)

# Define criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [10]:
# Main cross-validation loop
for fold, (train_idx, test_idx) in enumerate(kf.split(train_data)):
    print(f"Fold {fold + 1}/{k_folds}")

    # Create data loaders for training and testing
    train_loader = torch.utils.data.DataLoader(
        train_data, batch_size=32, sampler=torch.utils.data.SubsetRandomSampler(train_idx))
    test_loader = torch.utils.data.DataLoader(
        train_data, batch_size=32, sampler=torch.utils.data.SubsetRandomSampler(test_idx))

    # Train and test for each fold
    for epoch in range(10):  # Adjust number of epochs as needed
        train_loss = train(model, train_loader, criterion, optimizer, device)
        metrics = test(model, test_loader, criterion, device)
        # Print metrics
        print(f"Epoch {epoch + 1}/10")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Test Loss: {metrics['loss']:.4f}")
        print(f"Test Accuracy: {metrics['accuracy']:.4f}")
        print(f"Test Precision: {metrics['precision']:.4f}")
        print(f"Test Recall: {metrics['recall']:.4f}")
        print(f"Test F1 Score: {metrics['f1_score']:.4f}")

    # Save model for each fold if needed
    torch.save(model.state_dict(), f'darknet19_fold_{fold + 1}.pth')

Fold 1/5
Epoch 1/10
Train Loss: 0.7690
Test Loss: 0.2451
Test Accuracy: 0.5733
Test Precision: 0.6173
Test Recall: 0.5733
Test F1 Score: 0.5187


c:\Users\chari\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 2/10
Train Loss: 0.5303
Test Loss: 0.9382
Test Accuracy: 0.3631
Test Precision: 0.4442
Test Recall: 0.3631
Test F1 Score: 0.3091
Epoch 3/10
Train Loss: 0.4602
Test Loss: 0.2014
Test Accuracy: 0.7049
Test Precision: 0.6764
Test Recall: 0.7049
Test F1 Score: 0.6649
Epoch 4/10
Train Loss: 0.3957
Test Loss: 0.1211
Test Accuracy: 0.7633
Test Precision: 0.8132
Test Recall: 0.7633
Test F1 Score: 0.7641


c:\Users\chari\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 5/10
Train Loss: 0.3637
Test Loss: 1.1149
Test Accuracy: 0.2747
Test Precision: 0.2998
Test Recall: 0.2747
Test F1 Score: 0.2081
Epoch 6/10
Train Loss: 0.3611
Test Loss: 0.3540
Test Accuracy: 0.5936
Test Precision: 0.6388
Test Recall: 0.5936
Test F1 Score: 0.5641
Epoch 7/10
Train Loss: 0.3383
Test Loss: 0.5256
Test Accuracy: 0.4170
Test Precision: 0.6013
Test Recall: 0.4170
Test F1 Score: 0.3443
Epoch 8/10
Train Loss: 0.3110
Test Loss: 0.1530
Test Accuracy: 0.6988
Test Precision: 0.7731
Test Recall: 0.6988
Test F1 Score: 0.6926
Epoch 9/10
Train Loss: 0.2967
Test Loss: 0.1281
Test Accuracy: 0.7730
Test Precision: 0.8422
Test Recall: 0.7730
Test F1 Score: 0.7613
Epoch 10/10
Train Loss: 0.2952
Test Loss: 0.0763
Test Accuracy: 0.8587
Test Precision: 0.8716
Test Recall: 0.8587
Test F1 Score: 0.8564
Fold 2/5
Epoch 1/10
Train Loss: 0.2820
Test Loss: 0.2695
Test Accuracy: 0.5632
Test Precision: 0.6649
Test Recall: 0.5632
Test F1 Score: 0.5345
Epoch 2/10
Train Loss: 0.2605
Test Loss: 0.22

c:\Users\chari\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 5/10
Train Loss: 0.2247
Test Loss: 0.4306
Test Accuracy: 0.5234
Test Precision: 0.5861
Test Recall: 0.5234
Test F1 Score: 0.4816
Epoch 6/10
Train Loss: 0.2233
Test Loss: 0.1133
Test Accuracy: 0.8134
Test Precision: 0.8355
Test Recall: 0.8134
Test F1 Score: 0.8075
Epoch 7/10
Train Loss: 0.1950
Test Loss: 0.1269
Test Accuracy: 0.7949
Test Precision: 0.8477
Test Recall: 0.7949
Test F1 Score: 0.7813
Epoch 8/10
Train Loss: 0.1668
Test Loss: 0.0493
Test Accuracy: 0.9213
Test Precision: 0.9245
Test Recall: 0.9213
Test F1 Score: 0.9192
Epoch 9/10
Train Loss: 0.1746
Test Loss: 0.1296
Test Accuracy: 0.7975
Test Precision: 0.8323
Test Recall: 0.7975
Test F1 Score: 0.7885
Epoch 10/10
Train Loss: 0.1639
Test Loss: 0.0767
Test Accuracy: 0.8718
Test Precision: 0.8985
Test Recall: 0.8718
Test F1 Score: 0.8666
Fold 3/5
Epoch 1/10
Train Loss: 0.1415
Test Loss: 0.1877
Test Accuracy: 0.7754
Test Precision: 0.8152
Test Recall: 0.7754
Test F1 Score: 0.7731
Epoch 2/10
Train Loss: 0.1445
Test Loss: 0.09

KeyboardInterrupt: 